In [18]:

# Imports
import os
import requests
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model, model_selection
from tempfile import TemporaryDirectory
import seaborn as sns
import time
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.optim as optim
from torch.optim import lr_scheduler

import torchvision
from torchvision import transforms
from torchvision.utils import make_grid
from torchvision.models import resnet18
import numpy as np
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
print("Running on device:", DEVICE.upper())

# manual random seed is used for dataset partitioning
# to ensure reproducible results across runs
RNG = torch.Generator().manual_seed(1991)
torch.manual_seed(1991)
torch.cuda.manual_seed(1991)

import random
random.seed(1991)

import numpy as np
np.random.seed(1991)

Running on device: CUDA


In [19]:
# CIFAR 10 dataset
torch.manual_seed(1991)
# Transformations
normalize = transforms.Compose(
    [
        transforms.ToTensor(),
        # transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
    ]
)

# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=normalize
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)


Files already downloaded and verified
Files already downloaded and verified


In [20]:

def unlearning(
    net,
    retain_loader,
    forget_loader,
    val_loader):
    """Simple unlearning by finetuning."""



    class CustomCrossEntropyLoss(nn.Module):
        def __init__(self, class_weights=None):
            super(CustomCrossEntropyLoss, self).__init__()
            self.class_weights = class_weights

        def forward(self, input, target):
            # Compute the standard cross-entropy loss
            ce_loss = nn.functional.cross_entropy(input, target)

            # Apply class weights to the loss if provided
            if self.class_weights is not None:
                # Calculate the weights for each element in the batch based on the target
                weights = torch.tensor([self.class_weights[i] for i in target], device=input.device)
                ce_loss = torch.mean(ce_loss * weights)

            return ce_loss



    # Define the vision_confuser function
    def vision_confuser(model, std = 0.6):
        for name, module in model.named_modules():
            if hasattr(module, 'weight'):
                if 'conv' in name:
                    actual_value = module.weight.clone().detach()
                    new_values = torch.normal(mean=actual_value, std=std)
                    module.weight.data.copy_(new_values)

    vision_confuser(net)

    epochs = 4

    # w = 0.05

    # class_weights = [1, w, w, w, w, w, w, w, w, w]
    criterion = CustomCrossEntropyLoss(None)


    optimizer = optim.SGD(net.parameters(), lr=0.0007,
                      momentum=0.9, weight_decay=5e-4)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer, T_max=epochs)

    net.train()
    i=0

    for ep in range(epochs):
        i=0
        net.train()
        for sample in retain_loader:
            inputs = sample[0]
            targets = sample[1]
            inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

            optimizer.zero_grad()
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

        if (ep == epochs-2):
            vision_confuser(net , 0.005) # increase model robustness before last training epoch

        scheduler.step()

    net.eval()
    return net

In [21]:
from sklearn.linear_model import LogisticRegression

def train_mia_on_class(model, class_index, train_set, test_set, no_class=True, splits_num=5):
  model = model.to("cpu")

  if not no_class:
    # Select indencies where class is class_index
    train_class_set = np.where(np.array(train_set.targets) == class_index)[0]
    test_class_set = np.where(np.array(test_set.targets) == class_index)[0]
  else:
    train_class_set = np.arange(len(train_set))
    test_class_set = np.arange(len(test_set))

  max_len = min([len(train_class_set), len(test_class_set)])
  # Make equal sizes
  train_class_set = train_class_set[:max_len]
  test_class_set = test_class_set[:max_len]

  # Obtain subsets
  train_class_set = torch.utils.data.Subset(train_set, train_class_set)
  test_class_set = torch.utils.data.Subset(test_set, test_class_set)


  # Make them
  class_test_loader = torch.utils.data.DataLoader(
    test_class_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )

  class_train_loader = torch.utils.data.DataLoader(
    train_class_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )

  criterion = nn.CrossEntropyLoss(reduction="none")
  # Obtain train and test logits
  logits_train = []
  for i, (images, labels) in enumerate(class_train_loader, 0):
    for img, label in zip(images, labels):
      logits = model(img.unsqueeze(0))
      losses = criterion(logits, label.unsqueeze(0)).numpy(force=True)

      logits_train.append(np.concatenate((logits.detach().numpy()[0], losses)))

  logits_test = []
  for i, (images, labels) in enumerate(class_test_loader, 0):
    for img, label in zip(images, labels):
      logits = model(img.unsqueeze(0))
      losses = criterion(logits, label.unsqueeze(0)).numpy(force=True)

      logits_test.append((np.concatenate((logits.detach().numpy()[0], losses))))

  logits_train, logits_test = np.array(logits_train), np.array(logits_test)
  # Create dataset
  ys = [1] * max_len + [0] * max_len
  ys = np.array(ys)
  p = np.random.permutation(len(ys))
  logits = np.concatenate((logits_train, logits_test))

  logits = logits[p]
  ys = ys[p]

  # Fit logitstic regression
  clf = LogisticRegression(random_state=0, max_iter=1000)
  cv = model_selection.StratifiedShuffleSplit(
        n_splits=splits_num, random_state=0
    )

  clf = model_selection.cross_val_score(
        clf, logits, ys, cv=cv, scoring="accuracy"
    )

  # Output score
  # print(f"Mean MIA attack score for class {class_index} is: " + str(clf.mean()))
  return clf.mean()


In [22]:


# Check accuracy

def accuracy(net, loader):
    """Return accuracy on a dataset given by the data loader."""
    correct = 0
    total = 0
    for i, (inputs, targets) in enumerate(loader, 0):
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)
        outputs = net(inputs)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
    return correct / total


In [23]:

def compute_losses(net, loader):
    """Auxiliary function to compute per-sample losses"""

    criterion = nn.CrossEntropyLoss(reduction="none")
    all_losses = []

    for inputs, targets in loader:
        inputs, targets = inputs.to(DEVICE), targets.to(DEVICE)

        logits = net(inputs)
        losses = criterion(logits, targets).numpy(force=True)
        for l in losses:
            all_losses.append(l)

    return np.array(all_losses)

## Baseline

In [42]:


print("----- 5 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Baseline"
model_params = "cifar10_resnet18_baseline.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()
  model_forget_ft = model_forget_ft.to(DEVICE)

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_ft_forget.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)

----- 5 PERCENT -------
Model name: Resnet18 Baseline
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 97.6
MIA score before on forget set: 0.87


Retain set accuracy FORGET model: 30.6%
Test set accuracy FORGET model: 29.9%
Forget set accuracy FORGET model: 32.0%

The MIA has an accuracy of 0.660 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 96.8
MIA score before on forget set: 0.97


Retain set accuracy FORGET model: 31.0%
Test set accuracy FORGET model: 29.9%
Forget set accuracy FORGET model: 27.2%

The MIA has an accuracy of 0.684 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 93.2
MIA score before on forget set: 0.92


Retain set accuracy FORGET model: 31.8%
Test set accuracy FORGET model: 29.7%
Forget set accuracy FORGET model: 11.2%

The MIA has an accuracy of 0.628 on forgotten vs

In [43]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)


[0.87, 0.97, 0.92, 0.86, 0.98, 0.92, 0.94, 0.94, 0.91, 0.88]
[0.66, 0.6839999999999999, 0.628, 0.6799999999999999, 0.728, 0.708, 0.708, 0.668, 0.728, 0.708]
[97.6, 96.8, 93.2, 85.2, 94.8, 88.4, 96.8, 95.19999999999999, 98.4, 95.6]
[30.574592519645478, 31.004682758204876, 31.822018569878214, 31.516430509496534, 31.617558387265344, 30.593130062508795, 31.679864935482577, 32.17034789074904, 31.73697220603308, 30.776806351120488]
[32.0, 27.200000000000003, 11.200000000000001, 10.8, 32.0, 22.400000000000002, 35.199999999999996, 17.2, 41.199999999999996, 31.2]
[29.89, 29.89, 29.69, 29.880000000000003, 30.020000000000003, 28.51, 30.830000000000002, 31.78, 29.99, 29.67]


In [44]:

mean_mia_score_before_baseline = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_baseline = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_baseline = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_baseline = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_baseline = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_baseline = np.mean(np.array(Accuracy_test))

In [45]:


print(mean_mia_score_before_baseline)
print(mean_mia_score_after_baseline)

print(mean_accuracy_forget_before_baseline)

print(mean_accuracy_forget_after_baseline)
print(mean_accuracy_retain_baseline)
print(mean_accuracy_test_baseline)

0.9190000000000002
0.69
94.2
26.04
31.349240419038445
30.015000000000004


## Dropout model

In [46]:
def modify_model(model_train_reg):
  from collections import OrderedDict

  layer1 = model_train_reg.layer1

  for indx in range(len(layer1)):
    modified_layers = OrderedDict()
    for name, feature in layer1[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer1[indx] = nn.Sequential(modified_layers)



  layer2 = model_train_reg.layer2

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer2[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer2[indx] = nn.Sequential(modified_layers)

  layer3 = model_train_reg.layer3

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer3[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)
    model_train_reg.layer3[indx] = nn.Sequential(modified_layers)

  layer4 = model_train_reg.layer4

  for indx in [1]:
    modified_layers = OrderedDict()
    for name, feature in layer4[indx].named_children():
      modified_layers[name] = feature
      if isinstance(feature, nn.ReLU):
        modified_layers["dropout"] = nn.Dropout(p=0.3)

    model_train_reg.layer4[indx] = nn.Sequential(modified_layers)

  return model_train_reg

In [47]:


print("----- 5 PERCENT DROPOUT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Dropout"
model_params = "cifar10_resnet18_l2_dropout_regularization.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft = model_forget_ft.to(DEVICE)
  model_forget_ft = modify_model(model_forget_ft)

  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_ft_forget.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)

----- 5 PERCENT DROPOUT -------
Model name: Resnet18 Dropout
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 84.0
MIA score before on forget set: 0.93


Retain set accuracy FORGET model: 10.1%
Test set accuracy FORGET model: 10.1%
Forget set accuracy FORGET model: 0.0%

The MIA has an accuracy of 0.836 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 87.6
MIA score before on forget set: 0.96


Retain set accuracy FORGET model: 11.0%
Test set accuracy FORGET model: 11.0%
Forget set accuracy FORGET model: 0.0%

The MIA has an accuracy of 0.884 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 69.6
MIA score before on forget set: 0.89


Retain set accuracy FORGET model: 10.3%
Test set accuracy FORGET model: 10.5%
Forget set accuracy FORGET model: 0.0%

The MIA has an accuracy of 0.828 on forgotte

In [48]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.9299999999999999, 0.96, 0.89, 0.89, 0.89, 0.87, 0.84, 0.9299999999999999, 0.89, 0.86]
[0.836, 0.884, 0.828, 0.552, 0.9039999999999999, 0.656, 0.8240000000000001, 0.5840000000000001, 0.7959999999999999, 0.8800000000000001]
[84.0, 87.6, 69.6, 60.0, 76.4, 70.39999999999999, 85.2, 84.39999999999999, 87.6, 87.2]
[10.11958597125917, 10.973109851682143, 10.324590493417746, 10.177257928373328, 9.799823136908111, 10.453759897110244, 10.140092054750466, 10.065117774740735, 10.7160801495267, 10.443171540548688]
[0.0, 0.0, 0.0, 0.0, 96.0, 0.0, 2.0, 0.0, 0.0, 0.0]
[10.07, 11.020000000000001, 10.459999999999999, 10.07, 10.23, 10.489999999999998, 10.03, 10.09, 10.639999999999999, 10.15]


In [49]:


mean_mia_score_before_dropout = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_dropout = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_dropout = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_dropout = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_dropout = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_dropout = np.mean(np.array(Accuracy_test))

In [50]:


print(mean_mia_score_before_dropout)
print(mean_mia_score_after_dropout)

print(mean_accuracy_forget_before_dropout)

print(mean_accuracy_forget_after_dropout)
print(mean_accuracy_retain_dropout)
print(mean_accuracy_test_dropout)


0.8949999999999999
0.7744
79.24
9.8
10.321258879831735
10.325000000000001


## Augmentation model

In [51]:

# CIFAR 10 dataset
torch.manual_seed(1991)
# Transformations
normalize = transforms.Compose(
    [
         transforms.RandomRotation(30), # Randomly rotate some images by 20 degrees
         transforms.RandomHorizontalFlip(), # Randomly horizontal flip the images
         transforms.ColorJitter(brightness = 0.1, # Randomly adjust color jitter of the images
                                                            contrast = 0.1,
                                                            saturation = 0.1),
         transforms.RandomAdjustSharpness(sharpness_factor = 2,
                                                                      p = 0.1), # Randomly adjust sharpness
          transforms.RandomAffine(0, shear=10, scale=(0.8,1.2)), #Performs actions like zooms, change shear angles.
        transforms.ToTensor(),
    ]
)

to_tensor_transform = transforms.Compose(
    [
        transforms.ToTensor(),
    ]
)


# Train data
train_set = torchvision.datasets.CIFAR10(
    root="./data", train=True, download=True, transform=normalize
)
# Train loader
train_loader = DataLoader(train_set, batch_size=256, shuffle=True, num_workers=2)

# Test data
test_set = torchvision.datasets.CIFAR10(
    root="./data", train=False, download=True, transform=to_tensor_transform
)
# Test loader
test_loader = DataLoader(test_set, batch_size=256, shuffle=True, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [52]:


print("----- 5 PERCENT -------")
# Generalisation algorithm
# Calculating score across 10 different classes
# Model parameters
model_name = "Resnet18 Augmentation"
model_params = "cifar10_resnet18_augmentation.pt"

# Percantage of class to forget
amount = 0.05

# Show different classes
classes = np.unique(np.array(train_set.targets))

# mia scores
MIA_scores_before = []
MIA_scores = []

# Accuracy
Accuracy_forget_before = []

Accuracy_retain = []
Accuracy_forget = []
Accuracy_test = []


print(f"Model name: {model_name}")
print(f"Amount: {100 * amount}")

for class_num in classes:
  print(f"------ UNLEARNING CLASS {class_num} ------\n")
  # Define model from trained params

  model_forget_ft = resnet18(weights=None, num_classes=10) # Load resnet18 from pytorch
  model_forget_ft.load_state_dict(torch.load(model_params))
  model_forget_ft.eval()
  model_forget_ft = model_forget_ft.to(DEVICE)

  # --- FORMING DATASET ------
  # Choose random forget indecies from some class
  # Index of class
  class_index = class_num # cars
  class_set = np.where(np.array(train_set.targets) == class_num)[0]

  # Percantage of whole data ( from class )
  amount_int = class_set.shape[0] * amount

  # Get indeces
  forget_idx = np.random.choice(class_set, int(amount_int))

  # construct indices of retain from those of the forget set
  forget_mask = np.zeros(len(train_set.targets), dtype=bool)
  forget_mask[forget_idx] = True
  retain_idx = np.arange(forget_mask.size)[~forget_mask]

  # split train set into a forget and a retain set
  forget_set = torch.utils.data.Subset(train_set, forget_idx)
  retain_set = torch.utils.data.Subset(train_set, retain_idx)

  # Generate forget and retain loaders
  forget_loader = torch.utils.data.DataLoader(
      forget_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  retain_loader = torch.utils.data.DataLoader(
      retain_set, batch_size=256, shuffle=True, num_workers=2, generator=RNG
  )
  # ---------

  # --- EVALUATING MODEL BEFORE ----
  accuracy_forget = 100.0 * accuracy(model_forget_ft, forget_loader)
  Accuracy_forget_before.append(accuracy_forget)
  print(f"Accuracy before on forget set: {accuracy_forget}")

  mia_attack = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 2)
  MIA_scores_before.append(mia_attack)
  print(f"MIA score before on forget set: {np.round(mia_attack, 3)}")
  print("\n")

  model_forget_ft = model_forget_ft.to(DEVICE)
  # --------------

  # Unlearn
  model_ft_forget = unlearning(model_forget_ft, retain_loader, forget_loader, test_loader)

  # Compare accuracy
  print(f"Retain set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, retain_loader):0.1f}%")
  print(f"Test set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, test_loader):0.1f}%")
  print(f"Forget set accuracy FORGET model: {100.0 * accuracy(model_ft_forget, forget_loader):0.1f}%")

  Accuracy_retain.append(100.0 * accuracy(model_ft_forget, retain_loader))
  Accuracy_forget.append(100.0 * accuracy(model_ft_forget, forget_loader))
  Accuracy_test.append(100.0 * accuracy(model_ft_forget, test_loader))
  print()
  model_forget_ft.eval()
  mia_scores_fr = train_mia_on_class(model_forget_ft, class_num, forget_set, test_set, True, 5)
  MIA_scores.append(mia_scores_fr)
  print(
      f"The MIA has an accuracy of {mia_scores_fr:.3f} on forgotten vs unseen images on FORGET model"
  )
  print()
  print('-' * 20)


----- 5 PERCENT -------
Model name: Resnet18 Augmentation
Amount: 5.0
------ UNLEARNING CLASS 0 ------

Accuracy before on forget set: 83.6
MIA score before on forget set: 0.86


Retain set accuracy FORGET model: 24.0%
Test set accuracy FORGET model: 26.6%
Forget set accuracy FORGET model: 32.0%

The MIA has an accuracy of 0.708 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 1 ------

Accuracy before on forget set: 77.60000000000001
MIA score before on forget set: 0.96


Retain set accuracy FORGET model: 23.7%
Test set accuracy FORGET model: 26.2%
Forget set accuracy FORGET model: 14.8%

The MIA has an accuracy of 0.660 on forgotten vs unseen images on FORGET model

--------------------
------ UNLEARNING CLASS 2 ------

Accuracy before on forget set: 73.6
MIA score before on forget set: 0.83


Retain set accuracy FORGET model: 23.9%
Test set accuracy FORGET model: 26.4%
Forget set accuracy FORGET model: 10.4%

The MIA has an accuracy of 0.63

In [53]:

# mia scores
print(MIA_scores_before)
print(MIA_scores)

# Accuracy
print(Accuracy_forget_before)

print(Accuracy_retain)
print(Accuracy_forget)
print(Accuracy_test)

[0.86, 0.96, 0.83, 0.81, 0.85, 0.85, 0.9299999999999999, 0.86, 0.9299999999999999, 0.89]
[0.708, 0.66, 0.636, 0.6799999999999999, 0.7000000000000001, 0.676, 0.724, 0.7120000000000001, 0.644, 0.704]
[83.6, 77.60000000000001, 73.6, 66.4, 72.8, 64.4, 83.6, 78.8, 86.8, 79.2]
[24.196012220614247, 23.710179881418952, 24.160520868918674, 23.76961877775768, 24.147356150893344, 23.940831256531876, 23.87900713496131, 23.525392391326193, 23.45351501266128, 23.1545310207609]
[29.2, 19.2, 7.6, 9.6, 22.0, 18.8, 33.6, 16.0, 34.8, 26.400000000000002]
[26.57, 26.179999999999996, 26.41, 26.31, 26.450000000000003, 26.479999999999997, 25.86, 25.509999999999998, 25.900000000000002, 25.490000000000002]


In [54]:


mean_mia_score_before_aug = np.mean(np.array(MIA_scores_before))
mean_mia_score_after_aug = np.mean(np.array(MIA_scores))

mean_accuracy_forget_before_aug = np.mean(np.array(Accuracy_forget_before))

mean_accuracy_forget_after_aug = np.mean(np.array(Accuracy_forget))
mean_accuracy_retain_aug = np.mean(np.array(Accuracy_retain))
mean_accuracy_test_aug = np.mean(np.array(Accuracy_test))

In [55]:


print(mean_mia_score_before_aug)
print(mean_mia_score_after_aug)

print(mean_accuracy_forget_before_aug)

print(mean_accuracy_forget_after_aug)
print(mean_accuracy_retain_aug)
print(mean_accuracy_test_aug)


0.877
0.6844
76.67999999999999
21.720000000000002
23.793696471584447
26.115999999999996
